In [215]:
from gurobipy import Model, GRB

In [216]:
# Initialize the model
model = gp.Model("Optimal_Coffee_Ordering")


In [217]:
# Problem data
scenarios = [
    {'n': 1, 'p': 0.09, 'd': 90},
    {'n': 2, 'p': 0.12, 'd': 95},
    {'n': 3, 'p': 0.10, 'd': 100},
    {'n': 4, 'p': 0.05, 'd': 105},
    {'n': 5, 'p': 0.16, 'd': 110},
    {'n': 6, 'p': 0.14, 'd': 115},
    {'n': 7, 'p': 0.03, 'd': 120},
    {'n': 8, 'p': 0.08, 'd': 125},
    {'n': 9, 'p': 0.05, 'd': 130},
    {'n': 10, 'p': 0.05, 'd': 135},
    {'n': 11, 'p': 0.04, 'd': 140},
    {'n': 12, 'p': 0.03, 'd': 145},
    {'n': 13, 'p': 0.02, 'd': 150},
    {'n': 14, 'p': 0.01, 'd': 155},
    {'n': 15, 'p': 0.02, 'd': 160},
    {'n': 16, 'p': 0.01, 'd': 165}
]

In [218]:
# Define the decision variables
initial_order = model.addVar(name="InitialOrder", vtype=GRB.INTEGER)
phil_sebastian_order = {s['n']: model.addVar(name=f"PhilSebastian_{s['n']}", vtype=GRB.INTEGER) for s in scenarios}
rosso_order = {s['n']: model.addVar(name=f"Rosso_{s['n']}", vtype=GRB.INTEGER) for s in scenarios}
monogram_order = {s['n']: model.addVar(name=f"Monogram_{s['n']}", vtype=GRB.INTEGER) for s in scenarios}



In [219]:
# Set the objective to minimize the total expected cost
model.setObjective(95 * initial_order + sum(s['p'] * (120 * phil_sebastian_order[s['n']] + 105 * rosso_order[s['n']] + 110 * monogram_order[s['n']]) for s in scenarios), GRB.MINIMIZE)


In [220]:
for s in scenarios:
    model.addConstr(initial_order + phil_sebastian_order[s['n']] + rosso_order[s['n']] + monogram_order[s['n']] >= s['d'], name=f"SupplyRequirement_{s['n']}")

    # Add binary variables and constraints for minimum orders from Rosso and Monogram
    rosso_indicator = model.addVar(vtype=GRB.BINARY, name=f"RossoIndicator_{s['n']}")
    model.addGenConstrIndicator(rosso_indicator, True, rosso_order[s['n']] >= 70, name=f"RossoMinOrder_{s['n']}")
    model.addGenConstrIndicator(rosso_indicator, False, rosso_order[s['n']] == 0, name=f"NoRossoOrder_{s['n']}")

    monogram_indicator = model.addVar(vtype=GRB.BINARY, name=f"MonogramIndicator_{s['n']}")
    model.addGenConstrIndicator(monogram_indicator, True, monogram_order[s['n']] >= 40, name=f"MonogramMinOrder_{s['n']}")
    model.addGenConstrIndicator(monogram_indicator, False, monogram_order[s['n']] == 0, name=f"NoMonogramOrder_{s['n']}")


In [221]:
# Execute the model optimization
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.3.0 23D56)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 16 rows, 81 columns and 64 nonzeros
Model fingerprint: 0xdc6a3717
Model has 64 general constraints
Variable types: 0 continuous, 81 integer (32 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [9e+01, 2e+02]
  GenCon rhs range [4e+01, 7e+01]
  GenCon coe range [1e+00, 1e+00]
Found heuristic solution: objective 8.600000e+11
Presolve added 64 rows and 0 columns
Presolve time: 0.00s
Presolved: 80 rows, 81 columns, 192 nonzeros
Found heuristic solution: objective 11690.250000
Variable types: 0 continuous, 81 integer (32 binary)

Root relaxation: objective 1.113550e+04, 38 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work


In [236]:
# Output the optimized results
total_initial_cost = 95 * initial_order.X
print(f"Optimal initial order quantity: {initial_order.X}, Total Initial Cost: ${total_initial_cost:.2f}")
for s in scenarios:
    cost_phil = 120 * phil_sebastian_order[s['n']].X
    cost_rosso = 105 * rosso_order[s['n']].X
    cost_monogram = 110 * monogram_order[s['n']].X
    total_cost_local = cost_phil + cost_rosso + cost_monogram
    print(f"Scenario {s['n']}: Local Costs: Phil & Sebastian = ${cost_phil:.2f}, Rosso = ${cost_rosso:.2f}, Monogram = ${cost_monogram:.2f}, Total Local = ${total_cost_local:.2f}")

Optimal initial order quantity: 95.0, Total Initial Cost: $9025.00
Scenario 1: Local Costs: Phil & Sebastian = $0.00, Rosso = $0.00, Monogram = $0.00, Total Local = $0.00
Scenario 2: Local Costs: Phil & Sebastian = $0.00, Rosso = $0.00, Monogram = $0.00, Total Local = $0.00
Scenario 3: Local Costs: Phil & Sebastian = $600.00, Rosso = $0.00, Monogram = $0.00, Total Local = $600.00
Scenario 4: Local Costs: Phil & Sebastian = $1200.00, Rosso = $0.00, Monogram = $0.00, Total Local = $1200.00
Scenario 5: Local Costs: Phil & Sebastian = $1800.00, Rosso = $0.00, Monogram = $0.00, Total Local = $1800.00
Scenario 6: Local Costs: Phil & Sebastian = $2400.00, Rosso = $0.00, Monogram = $0.00, Total Local = $2400.00
Scenario 7: Local Costs: Phil & Sebastian = $3000.00, Rosso = $0.00, Monogram = $0.00, Total Local = $3000.00
Scenario 8: Local Costs: Phil & Sebastian = $3600.00, Rosso = $0.00, Monogram = $0.00, Total Local = $3600.00
Scenario 9: Local Costs: Phil & Sebastian = $4200.00, Rosso = $0.00

In [237]:
#E
for s in scenarios:
    # Add binary variables and constraints for minimum orders from Rosso
    rosso_indicator = model.addVar(vtype=GRB.BINARY, name=f"RossoIndicator_{s['n']}")
    model.addGenConstrIndicator(rosso_indicator, True, rosso_order[s['n']] >= 70, name=f"RossoMinOrder_{s['n']}")
    model.addGenConstrIndicator(rosso_indicator, False, rosso_order[s['n']] == 0, name=f"NoRossoOrder_{s['n']}")


In [225]:
##F
# After optimization
model.optimize()

# Retrieve the optimal number of gallons to order initially
optimal_gallons = initial_order.X

# Calculate the total initial cost based on the optimal initial order quantity
total_initial_cost = 95 * optimal_gallons

# Retrieve the optimal objective function value (total expected cost including scenarios)
optimal_total_cost = model.ObjVal

print(f"Optimal gallons to order in advance: {optimal_gallons}")
print(f"Total Initial Cost: ${total_initial_cost:.2f}")
print(f"Optimal Total Expected Cost: ${optimal_total_cost:.2f}")


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.3.0 23D56)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 16 rows, 113 columns and 64 nonzeros
Model fingerprint: 0xddff030f
Model has 128 general constraints
Variable types: 0 continuous, 113 integer (64 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [9e+01, 2e+02]
  GenCon rhs range [4e+01, 7e+01]
  GenCon coe range [1e+00, 1e+00]

MIP start from previous solve produced solution with objective 11343.5 (0.00s)
Loaded MIP start from previous solve with objective 11343.5

Presolve added 64 rows and 0 columns
Presolve removed 0 rows and 32 columns
Presolve time: 0.00s
Presolved: 80 rows, 81 columns, 192 nonzeros
Variable types: 0 continuous, 81 integer (32 binary)

Root relaxation: objective 1.113550e+04, 38 iterations, 0.00 seconds (0.00 w

In [238]:
#G
# Calculate the total expected cost of supplying all demand from local suppliers
expected_cost_without_initial_order = sum(s['p'] * min(120 * s['d'], 105 * max(s['d'] - 70, 0) + 70 * 105, 110 * max(s['d'] - 40, 0) + 40 * 110) for s in scenarios)
# Calculate total expected demand
total_demand = sum(s['p'] * s['d'] for s in scenarios)
# Calculate the threshold price
threshold_price_no_order = expected_cost_without_initial_order / total_demand
print(f"Threshold price for no initial order: ${round(threshold_price_no_order, 1)} per gallon")


Threshold price for no initial order: $105.0 per gallon


In [233]:
##H
max_demand = max(s['d'] for s in scenarios)  # The maximum demand from the scenarios
cost_if_max_ordered = 95 * max_demand
threshold_price_max_order = cost_if_max_ordered / total_demand
print(f"Threshold price for maximum order: ${round(threshold_price_max_order, 1)} per gallon")


Threshold price for maximum order: $136.7 per gallon


In [239]:
# I 
# Assume perfect_info_costs are calculated by simulating the minimum cost for each scenario
perfect_info_costs = [min(95 * s['d'], 120 * s['d'], 105 * max(s['d'] - 70, 0) + 70 * 105, 110 * max(s['d'] - 40, 0) + 40 * 110) for s in scenarios]
perfect_information_cost = sum(s['p'] * cost for s, cost in zip(scenarios, perfect_info_costs))

# EVPI
evpi = model.ObjVal - perfect_information_cost
print(f"Expected Value of Perfect Information (EVPI): ${evpi:.2f}")

Expected Value of Perfect Information (EVPI): $451.75
